In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (optional)
chrome_options.add_argument("--window-size=1920x1080")  # Set window size (optional, useful for visibility)

# Initialize the WebDriver with the headless options
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open the target URL
driver.get('https://www.wunderground.com/dashboard/pws/KGAATLAN216/table/2022-01-1/2022-01-1/daily')

# Wait for the table body element to be present in the DOM
tbody = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//html/body/app-root/app-dashboard/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/section[1]/div[1]/div/section/div/div/div/lib-history/div[2]/lib-history-table/div/div/div/table'))
)

# Extract data from the table
dataframe = []
table_rows = tbody.find_elements(By.XPATH, './/tr')

for row in table_rows:
    row_data = [cell.text for cell in row.find_elements(By.XPATH, './/td')]
    dataframe.append(row_data)

# Convert the data into a pandas DataFrame
df = pd.DataFrame(dataframe)
print(df)

# Close the browser
driver.quit()


           0        1        2     3     4        5     6         7        8   \
0        None     None     None  None  None     None  None      None     None   
1        None     None     None  None  None     None  None      None     None   
2    12:06 AM  68.9 °F  66.0 °F  90 %   WSW  0.0 mph    --  29.82 in  0.00 in   
3    12:11 AM  68.9 °F  66.0 °F  90 %   WSW  0.0 mph    --  29.82 in  0.00 in   
4    12:18 AM  68.9 °F  66.0 °F  90 %    SW  0.0 mph    --  29.82 in  0.00 in   
..        ...      ...      ...   ...   ...      ...   ...       ...      ...   
214  11:33 PM  72.7 °F  66.0 °F  79 %    SW  2.0 mph    --  29.62 in  0.04 in   
215  11:39 PM  72.6 °F  66.0 °F  79 %   WSW  2.0 mph    --  29.62 in  0.00 in   
216  11:45 PM  72.7 °F  66.0 °F  78 %  West  2.0 mph    --  29.62 in  0.00 in   
217  11:51 PM  72.7 °F  66.0 °F  79 %   WSW  0.0 mph    --  29.62 in  0.00 in   
218  11:57 PM  72.7 °F  66.0 °F  79 %    SW  0.0 mph    --  29.62 in  0.00 in   

          9     10      11 

In [3]:
df = df.drop([0, 1])
df.index = df.index - 2

column_labels = [
    "Hour", "Temperature", "Dew Point", "Humidity", "Wind", 
    "Speed", "Gust", "Pressure", "Precip.Rate.", "Precip.Accum.", 
    "UV", "Solar"
]

# Set the column labels
df.columns = column_labels

for column in df.columns:
    if column not in ["Hour", "Wind"]:
        df[column] = df[column].str.replace(r'[^\d.]', '', regex=True)
        
df = df.round(1)


In [5]:
test_df = df.copy()

In [6]:
import pandas as pd

# Assuming `test_df` is already defined and contains your data

# Convert 'Temperature' column from Fahrenheit to Celsius
test_df['Temperature'] = pd.to_numeric(test_df['Temperature'], errors='coerce')
test_df['Temperature'] = (test_df['Temperature'] - 32) * 5.0 / 9.0

# Convert 'Dew Point' column from Fahrenheit to Celsius
test_df['Dew Point'] = pd.to_numeric(test_df['Dew Point'], errors='coerce')
test_df['Dew Point'] = (test_df['Dew Point'] - 32) * 5.0 / 9.0

# Round temperature and dew point values to 1 decimal place
test_df = test_df.round(1)

# Define wind direction to degrees mapping
wind_to_degrees = {
    'North': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'East': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'South': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'West': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5
}

# Convert 'Wind' column from directions to degrees
test_df['Wind'] = test_df['Wind'].map(wind_to_degrees)

# Convert 'Speed' column to numeric and handle non-numeric values
test_df['Speed'] = pd.to_numeric(test_df['Speed'], errors='coerce')

# Fill any NaN values in 'Speed' with 0 or drop them if necessary
test_df['Speed'] = test_df['Speed'].fillna(0)  # Use .dropna() if you prefer to remove rows

# Convert wind speed from m/s to km/h
conversion_factor = 0.44704
test_df['Speed'] = test_df['Speed'] * conversion_factor

# Convert 'Pressure', 'Precip.Rate.', and 'Precip.Accum.' from inches to meters
inch_to_meter = 0.0254
test_df['Pressure'] = pd.to_numeric(test_df['Pressure'], errors='coerce') * inch_to_meter
test_df['Precip.Rate.'] = pd.to_numeric(test_df['Precip.Rate.'], errors='coerce') * inch_to_meter
test_df['Precip.Accum.'] = pd.to_numeric(test_df['Precip.Accum.'], errors='coerce') * inch_to_meter

In [7]:
test_df['Hour'] = pd.to_datetime(test_df['Hour'], format='%I:%M %p').dt.time

In [8]:
# Define a function to compute and replace averages
def compute_and_replace_averages(df, start_index, end_index):
    columns = ['Temperature', 'Dew Point', 'Humidity', 'Wind', 'Speed', 'Pressure', 'Precip.Rate.', 'Precip.Accum.', 'UV', 'Solar']
    if start_index < end_index:  # Ensure valid range
        averages = df.loc[start_index:end_index, columns].mean()
        # Convert averages to a DataFrame to align with the original DataFrame's shape
        averages_df = pd.DataFrame([averages] * (end_index - start_index), columns=columns, index=df.index[start_index:end_index])
        # Replace values from start_index to end_index
        df.loc[start_index:end_index, columns] = averages_df

# Convert columns to numeric, forcing errors to NaN
numeric_columns = ['Temperature', 'Dew Point', 'Humidity', 'Speed', 'Pressure', 'Precip.Rate.', 'Precip.Accum.', 'UV', 'Solar']
for column in numeric_columns:
    test_df[column] = pd.to_numeric(test_df[column], errors='coerce')

# Define the time ranges for 24 hours
time_ranges = {}
for hour in range(24):
    start_time = pd.to_datetime(f'{hour}:00 AM', format='%H:%M %p').time()
    end_time = pd.to_datetime(f'{hour}:59 AM', format='%H:%M %p').time()
    time_ranges[f'{hour:02d}:00 AM - {hour:02d}:59 AM'] = (start_time, end_time)

# Initialize start index
start_index = 0

# Iterate through each time range and apply the function
for period, (start_time, end_time) in time_ranges.items():
    # Filter rows for the current time range
    time_range_rows = test_df[(test_df['Hour'] >= start_time) & (test_df['Hour'] <= end_time)]
    
    if not time_range_rows.empty:
        # Count the number of rows in the current time range
        period_count = time_range_rows.shape[0]
        # Compute averages and replace values for the current time range
        compute_and_replace_averages(test_df, start_index, start_index + period_count)
        # Update the start_index for the next period
        start_index += period_count


In [9]:
test_df = test_df.dropna()
test_df

,Hour,Temperature,Dew Point,Humidity,Wind,Speed,Gust,Pressure,Precip.Rate.,Precip.Accum.,UV,Solar
0,00:06:00,20.555556,18.487500,88.125000,197.5,0.894080,,0.758246,0.000000,0.00000,0.0,0.0
1,00:11:00,20.555556,18.487500,88.125000,197.5,0.894080,,0.758246,0.000000,0.00000,0.0,0.0
2,00:18:00,20.555556,18.487500,88.125000,197.5,0.894080,,0.758246,0.000000,0.00000,0.0,0.0
3,00:23:00,20.555556,18.487500,88.125000,197.5,0.894080,,0.758246,0.000000,0.00000,0.0,0.0
4,00:29:00,20.555556,18.487500,88.125000,197.5,0.894080,,0.758246,0.000000,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
212,23:33:00,22.666667,18.833333,78.333333,237.5,0.496711,,0.752348,0.000564,0.02636,0.0,0.0
213,23:39:00,22.666667,18.833333,78.333333,237.5,0.496711,,0.752348,0.000564,0.02636,0.0,0.0
214,23:45:00,22.666667,18.833333,78.333333,237.5,0.496711,,0.752348,0.000564,0.02636,0.0,0.0
215,23:51:00,22.666667,18.833333,78.333333,237.5,0.496711,,0.752348,0.000564,0.02636,0.0,0.0


In [10]:
df = test_df.copy()

In [11]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (optional)
chrome_options.add_argument("--window-size=1920x1080")  # Set window size (optional, useful for visibility)

# Initialize the WebDriver with the headless options
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open the target URL
driver.get('https://www.wunderground.com/dashboard/pws/KGAATLAN216/table/2022-01-2/2022-01-2/daily')

# Wait for the table body element to be present in the DOM
tbody = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//html/body/app-root/app-dashboard/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/section[1]/div[1]/div/section/div/div/div/lib-history/div[2]/lib-history-table/div/div/div/table'))
)

# Extract data from the table
dataframe = []
table_rows = tbody.find_elements(By.XPATH, './/tr')

for row in table_rows:
    row_data = [cell.text for cell in row.find_elements(By.XPATH, './/td')]
    dataframe.append(row_data)

# Convert the data into a pandas DataFrame
temp_df = pd.DataFrame(dataframe)
print(temp_df)

# Close the browser
driver.quit()


           0        1        2     3     4        5     6         7        8   \
0        None     None     None  None  None     None  None      None     None   
1        None     None     None  None  None     None  None      None     None   
2    12:03 AM  72.6 °F  66.0 °F  79 %    SE  0.0 mph    --  29.62 in  0.00 in   
3    12:09 AM  72.7 °F  66.0 °F  79 %  West  2.0 mph    --  29.62 in  0.00 in   
4    12:15 AM  72.7 °F  66.0 °F  79 %    SW  0.0 mph    --  29.62 in  0.00 in   
..        ...      ...      ...   ...   ...      ...   ...       ...      ...   
234  11:32 PM  53.1 °F  51.0 °F  93 %  West  0.0 mph    --  29.58 in  0.00 in   
235  11:37 PM  52.6 °F  51.0 °F  93 %        1.0 mph    --  29.56 in  0.00 in   
236  11:43 PM  52.0 °F  50.0 °F  93 %   NNW  0.0 mph    --  29.55 in  0.00 in   
237  11:50 PM  51.7 °F  50.0 °F  93 %    NW  0.0 mph    --  29.57 in  0.00 in   
238  11:55 PM  51.3 °F  49.0 °F  93 %    NE  0.0 mph    --  29.57 in  0.00 in   

          9     10      11 

In [12]:
# Drop rows and adjust index
temp_df = temp_df.drop([0, 1])
temp_df.index = temp_df.index - 2

# Set column labels
column_labels = [
    "Hour", "Temperature", "Dew Point", "Humidity", "Wind", 
    "Speed", "Gust", "Pressure", "Precip.Rate.", "Precip.Accum.", 
    "UV", "Solar"
]
temp_df.columns = column_labels

# Clean non-numeric characters and convert columns to numeric
for column in df.columns:
    if column not in ["Hour", "Wind"]:
        temp_df[column] = temp_df[column].str.replace(r'[^\d.]', '', regex=True)
        
temp_df = temp_df.round(1)
temp_df['Temperature'] = pd.to_numeric(temp_df['Temperature'], errors='coerce')
temp_df['Dew Point'] = pd.to_numeric(temp_df['Dew Point'], errors='coerce')

# Convert Fahrenheit to Celsius
temp_df['Temperature'] = (temp_df['Temperature'] - 32) * 5.0 / 9.0
temp_df['Dew Point'] = (temp_df['Dew Point'] - 32) * 5.0 / 9.0

# Convert wind directions to degrees
wind_to_degrees = {
    'North': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5, 'East': 90, 
    'ESE': 112.5, 'SE': 135, 'SSE': 157.5, 'South': 180, 
    'SSW': 202.5, 'SW': 225, 'WSW': 247.5, 'West': 270, 
    'WNW': 292.5, 'NW': 315, 'NNW': 337.5
}
temp_df['Wind'] = temp_df['Wind'].map(wind_to_degrees)
temp_df['Speed'] = pd.to_numeric(temp_df['Speed'], errors='coerce').fillna(0)

# Convert wind speed from m/s to km/h
conversion_factor = 3.6
temp_df['Speed'] = temp_df['Speed'] * conversion_factor

# Convert pressure and precipitation from inches to meters
inch_to_meter = 0.0254
temp_df['Pressure'] = pd.to_numeric(temp_df['Pressure'], errors='coerce') * inch_to_meter
temp_df['Precip.Rate.'] = pd.to_numeric(temp_df['Precip.Rate.'], errors='coerce') * inch_to_meter
temp_df['Precip.Accum.'] = pd.to_numeric(temp_df['Precip.Accum.'], errors='coerce') * inch_to_meter
temp_df['Hour'] = pd.to_datetime(temp_df['Hour'], format='%I:%M %p').dt.time

# Define a function to compute and replace averages
def compute_and_replace_averages(df, start_index, end_index):
    columns = ['Temperature', 'Dew Point', 'Humidity', 'Wind', 'Speed', 'Pressure', 'Precip.Rate.', 'Precip.Accum.', 'UV', 'Solar']
    if start_index < end_index:  # Ensure valid range
        averages = df.loc[start_index:end_index, columns].mean()
        averages_df = pd.DataFrame([averages] * (end_index - start_index), columns=columns, index=df.index[start_index:end_index])
        df.loc[start_index:end_index, columns] = averages_df

# Convert columns to numeric
numeric_columns = ['Temperature', 'Dew Point', 'Humidity', 'Speed', 'Pressure', 'Precip.Rate.', 'Precip.Accum.', 'UV', 'Solar']
for column in numeric_columns:
    temp_df[column] = pd.to_numeric(temp_df[column], errors='coerce')

# Define time ranges
time_ranges = {f'{hour:02d}:00 AM - {hour:02d}:59 AM': (pd.to_datetime(f'{hour}:00 AM', format='%H:%M %p').time(), 
                                                        pd.to_datetime(f'{hour}:59 AM', format='%H:%M %p').time()) 
               for hour in range(24)}

# Initialize start index
start_index = 0

# Iterate through each time range and apply the function
for period, (start_time, end_time) in time_ranges.items():
    time_range_rows = temp_df[(temp_df['Hour'] >= start_time) & (temp_df['Hour'] <= end_time)]
    if not time_range_rows.empty:
        period_count = time_range_rows.shape[0]
        compute_and_replace_averages(temp_df, start_index, start_index + period_count)
        start_index += period_count



In [13]:
temp_df.index = temp_df.index + len(test_df)
test_df = pd.concat([test_df, temp_df])

In [14]:
df = test_df.copy()

In [208]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime, timedelta
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from joblib import Parallel, delayed
import multiprocessing

# Function to scrape data for a single day
def scrape_data_for_date(date_str):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
    chrome_options.add_argument("--window-size=1920x1080")  # Set window size
    
    # Initialize the WebDriver with the headless options
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    # Construct the URL with the current date
    url = f"https://www.wunderground.com/dashboard/pws/KGAATLAN216/table/{date_str}/{date_str}/daily"
    
    # Open the URL
    driver.get(url)
    
    # Wait for the table body element to be present in the DOM
    tbody = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//html/body/app-root/app-dashboard/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/section[1]/div[1]/div/section/div/div/div/lib-history/div[2]/lib-history-table/div/div/div/table/tbody'))
    )
    
    # Extract data from the table
    dataframe = []
    table_rows = tbody.find_elements(By.XPATH, './/tr')

    for row in table_rows:
        row_data = [cell.text for cell in row.find_elements(By.XPATH, './/td')]
        dataframe.append(row_data)

    # Convert the data into a DataFrame for the current page
    new_df = pd.DataFrame(dataframe)
    
    # Close the browser
    driver.quit()
    
    print(f"Day {date_str} is done")
    
    return new_df

# Define the start and end dates
start_date = datetime.strptime("2023-06-01", "%Y-%m-%d")
end_date = datetime.strptime("2023-06-15", "%Y-%m-%d")

# Create a list of all dates to process
dates = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
date_strings = [date.strftime("%Y-%m-%d") for date in dates]

# Use joblib to parallelize the scraping process
num_cores = multiprocessing.cpu_count()

# Scrape data in parallel
results = Parallel(n_jobs=num_cores)(delayed(scrape_data_for_date)(date_str) for date_str in date_strings)

# Concatenate all the DataFrames together
all_data = pd.concat(results, ignore_index=True)

print(all_data)

            0        1        2     3     4        5   6         7        8   \
0     12:00 AM  72.7 °F  63.0 °F  72 %   SSW  0.0 mph  --  29.75 in  0.00 in   
1     12:06 AM  72.4 °F  63.0 °F  73 %   WSW  0.0 mph  --  29.75 in  0.00 in   
2     12:12 AM  72.2 °F  64.0 °F  74 %   WSW  0.0 mph  --  29.75 in  0.00 in   
3     12:18 AM  72.0 °F  64.0 °F  75 %    SW  0.0 mph  --  29.74 in  0.00 in   
4     12:24 AM  71.9 °F  64.0 °F  75 %  West  0.0 mph  --  29.74 in  0.00 in   
...        ...      ...      ...   ...   ...      ...  ..       ...      ...   
3651  11:31 PM  79.5 °F  67.0 °F  66 %    SW  0.0 mph  --  29.94 in  0.00 in   
3652  11:37 PM  79.3 °F  67.0 °F  67 %   WSW  0.0 mph  --  29.94 in  0.00 in   
3653  11:43 PM  79.1 °F  67.0 °F  67 %  West  0.0 mph  --  29.94 in  0.00 in   
3654  11:49 PM  79.0 °F  68.0 °F  68 %   WSW  0.0 mph  --  29.94 in  0.00 in   
3655  11:54 PM  78.9 °F  67.0 °F  67 %    SW  0.0 mph  --  29.93 in  0.00 in   

           9  10      11  
0     0.00 i

In [211]:
temp_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,12:00 AM,56.8 °F,55.0 °F,93 %,West,0.0 mph,--,29.91 in,0.00 in,0.00 in,0,0 w/m²
1,12:06 AM,56.8 °F,55.0 °F,93 %,SW,0.0 mph,--,29.92 in,0.00 in,0.00 in,0,0 w/m²
2,12:12 AM,56.8 °F,55.0 °F,93 %,WSW,0.0 mph,--,29.91 in,0.00 in,0.00 in,0,0 w/m²
3,12:18 AM,56.7 °F,55.0 °F,93 %,WSW,0.0 mph,--,29.92 in,0.00 in,0.00 in,0,0 w/m²
4,12:24 AM,56.7 °F,55.0 °F,93 %,SW,0.0 mph,--,29.92 in,0.00 in,0.00 in,0,0 w/m²
...,...,...,...,...,...,...,...,...,...,...,...,...
31914,11:31 PM,79.5 °F,67.0 °F,66 %,SW,0.0 mph,--,29.94 in,0.00 in,0.00 in,0,0 w/m²
31915,11:37 PM,79.3 °F,67.0 °F,67 %,WSW,0.0 mph,--,29.94 in,0.00 in,0.00 in,0,0 w/m²
31916,11:43 PM,79.1 °F,67.0 °F,67 %,West,0.0 mph,--,29.94 in,0.00 in,0.00 in,0,0 w/m²
31917,11:49 PM,79.0 °F,68.0 °F,68 %,WSW,0.0 mph,--,29.94 in,0.00 in,0.00 in,0,0 w/m²


In [209]:
all_data.index = all_data.index + len(temp_df)
temp_df = pd.concat([temp_df, all_data])

In [210]:
backup = temp_df.copy()